In [3]:
import socket
import os

def receive_image():
    # Create a TCP/IP socket
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # Bind the socket to the address of your computer and port 8888
    server_address = ('', 8888)
    server_socket.bind(server_address)

    # Listen for incoming connections
    server_socket.listen(1)

    print('Waiting for connection...')
    connection, client_address = server_socket.accept()

    try:
        print('Connection from', client_address)

        # Receive the image data
        image_data = b''
        while True:
            data = connection.recv(1024)
            if not data:
                break
            image_data += data

        # Write the image data to a file
        with open('received_image.jpg', 'wb') as f:
            f.write(image_data)

        print('Image received and saved as received_image.jpg')

    finally:
        # Clean up the connection
        connection.close()

if __name__ == '__main__':
    receive_image()


Waiting for connection...


In [ ]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install deepface


  Using cached protobuf-3.19.6-cp39-cp39-win_amd64.whl.metadata (807 bytes)
Using cached protobuf-3.19.6-cp39-cp39-win_amd64.whl (895 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.2 requires protobuf>=4.21.6, but you have protobuf 3.19.6 which is incompatible.


In [2]:
import firebase_admin
from firebase_admin import credentials
cred = credentials.Certificate("security-system-95cda-firebase-adminsdk-5wvap-938e03abfb.json")
firebase_admin.initialize_app(cred,{
    'databaseURL': 'https://security-system-95cda-default-rtdb.firebaseio.com/'})


In [ ]:
from deepface import DeepFace

In [ ]:
from firebase_admin import db
from datetime import datetime
import requests
import os
import random
import string

# Initialize Firebase Admin SDK # Update with your own service account key


# Function to write data to Firebase
def write_to_firebase(data):
    ref = db.reference('/entries')  # Reference to your Firebase database node
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    data['timestamp'] = timestamp  # Add timestamp to the data
    ref.push(data)  # Push data to Firebase

# Function to download image from ESP32 Cam
def download_image(url, save_directory=""):
    # Make HTTP request to get the image
    response = requests.get(url)
    if response.status_code == 200:
        # Generate a random filename
        random_filename = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
        # Get the file extension from the URL
        file_extension = "jpg"
        # Concatenate the random filename with the file extension
        filename = f"{random_filename}.{file_extension}"
        # Define the full path to save the image
        save_path = os.path.join(save_directory, filename)
        
        # Save the image to the specified directory
        with open(save_path, 'wb') as f:
            f.write(response.content)
        
        print(f"Image downloaded and saved as {save_path}")
        return save_path
    else:
        print("Failed to download image")
        return None

# ---------------------------- Handle Found Person ----------------------------
def takeAction(found_person):
    print("Found person:", found_person['name'])
    # Write the recognized person's information to Firebase
    write_to_firebase(found_person)

################################# Main Loop #################################
ESP_URL = "http://192.168.1.236" 
frame_path = download_image(ESP_URL)
if frame_path:
    # Perform face recognition
    recognition = DeepFace.find(img_path=frame_path, db_path="me/")
    
    if recognition:
        # Assuming recognition is a list of dictionaries with 'identity' keys
        for person in recognition:
            # Call takeAction function to handle found person
            takeAction(person)
    else:
        print("No match found.")
else:
    print("No frame downloaded.")


ConnectionError: HTTPConnectionPool(host='192.168.1.236', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002D9E02D3C40>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [3]:
import requests
import os
import random
import string
import logging
import paho.mqtt.client as mqtt
from firebase_admin import db
from datetime import datetime
# import tensorflow as tf
from deepface import DeepFace

# Configure logging
logging.basicConfig(level=logging.INFO)

# MQTT Broker details
mqtt_broker = "192.168.1.190"
mqtt_topic = "esp32/servo"
trigger_topic = "esp32/trigger"

# MQTT Client Setup
client = mqtt.Client()

def download_image(url, save_directory=""):
    try:
        response = requests.get(url)
        response.raise_for_status()

        random_filename = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
        file_extension = "jpg"
        filename = f"{random_filename}.{file_extension}"
        save_path = os.path.join(save_directory, filename)

        with open(save_path, 'wb') as f:
            f.write(response.content)

        logging.info(f"Image downloaded and saved as {save_path}")
        return save_path
    except requests.RequestException as e:
        logging.error(f"Failed to download image: {e}")
        return None

def take_action(found_person):
    logging.info(f"Found person: {found_person['identity']}")
    write_to_firebase(found_person)
    # Publish message to move the servo motor
    client.publish(mqtt_topic, "move_servo")
    
def write_to_firebase():
    ref = db.reference('/date/')
    date=ref.child('date')  # Reference to your Firebase database node
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
      # Add timestamp to the data
    date.push(timestamp)  # Push data to Firebase
def main():
    ESP_URL = os.getenv("ESP_URL", "http://192.168.1.236")
    SAVE_DIR = os.getenv("SAVE_DIR", "")

    frame_path = download_image(ESP_URL, SAVE_DIR)
    if frame_path:
        try:
            recognition = DeepFace.find(img_path=frame_path, db_path="me/")
            if recognition:
                for person in recognition:
                    take_action(person)
            else:
                logging.info("No match found.")
        except Exception as e:
            logging.error(f"Face recognition failed: {e}")
        finally:
            write_to_firebase()
            print('gg we lost a soldier')
    else:
        logging.info("No frame downloaded.")

def on_message(client, userdata, message):
    if message.payload.decode() == "start_script":
        main()

client.on_message = on_message
client.connect(mqtt_broker, 1883, 60)
client.subscribe(trigger_topic)
client.loop_start()

# Keep the script running to listen for MQTT messages
try:
    while True:
        pass
except KeyboardInterrupt:
    client.loop_stop()
    client.disconnect()


INFO:root:Image downloaded and saved as 9ogN54Otnuy4Oy9n.jpg


24-05-16 11:55:08 - Searching 9ogN54Otnuy4Oy9n.jpg in 12 length datastore


ERROR:root:Face recognition failed: Face could not be detected in 9ogN54Otnuy4Oy9n.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


gg we lost a soldier


INFO:root:Image downloaded and saved as OpkR0rSuzO2g0xey.jpg
ERROR:root:Face recognition failed: Face could not be detected in OpkR0rSuzO2g0xey.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


24-05-16 11:55:11 - Searching OpkR0rSuzO2g0xey.jpg in 12 length datastore
gg we lost a soldier


INFO:root:Image downloaded and saved as YNRZqLbN2A8Jy6ZO.jpg
ERROR:root:Face recognition failed: Face could not be detected in YNRZqLbN2A8Jy6ZO.jpg.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.


24-05-16 11:55:15 - Searching YNRZqLbN2A8Jy6ZO.jpg in 12 length datastore
gg we lost a soldier


In [ ]:
%pip install firebase_admin

INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/126.3 kB ? eta -:--:--
   --------- ------------------------------ 30.7/126.3 kB 1.3 MB/s eta 0:00:01
   ------------------------- -------------- 81.9/126.3 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 126.3/126.3 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/139.0 kB ? eta -:--:--
   -------------------------------------- - 133.1/139.0 kB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 139.0/139.0 kB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.6 MB 3.8 MB/s eta 0:00:03
    --------------------------------------- 0.3/11.6 MB 4.1 MB/s eta 0:00:03
   - -------------------------------------- 0.4/11.6 MB 4.2 MB/s eta 0:00:03
   - ----

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.25.3 which is incompatible.


In [ ]:

import requests
import os
import random
import string
import logging
import paho.mqtt.client as mqtt
from deepface import DeepFace
import pandas as pd

# Configure logging
logging.basicConfig(level=logging.INFO)

# MQTT Broker details
mqtt_broker = "192.168.1.190" # Replace with the IP address of your MQTT broker
mqtt_topic = "esp32/servo"
trigger_topic = "esp32/trigger"

# MQTT Client Setup
client = mqtt.Client()

def download_image(url, save_directory=""):
    try:
        response = requests.get(url)
        response.raise_for_status()

        random_filename = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
        file_extension = "jpg"
        filename = f"{random_filename}.{file_extension}"
        save_path = os.path.join(save_directory, filename)

        with open(save_path, 'wb') as f:
            f.write(response.content)

        logging.info(f"Image downloaded and saved as {save_path}")
        return save_path
    except requests.RequestException as e:
        logging.error(f"Failed to download image: {e}")
        return None

def take_action(found_person):
    logging.info(f"Found person: {found_person['identity']}")
    # Publish message to move the servo motor
    client.publish(mqtt_topic, "move_servo")

def main():
    ESP_URL = os.getenv("ESP_URL", "http://192.168.1.236")
    SAVE_DIR = os.getenv("SAVE_DIR", "")

    frame_path = download_image(ESP_URL, SAVE_DIR)
    if frame_path:
        try:
            models = [
                    "VGG-Face", 
                    "Facenet", 
                    "Facenet512", 
                    "OpenFace", 
                    "DeepFace", 
                    "DeepID", 
                    "ArcFace", 
                    "Dlib", 
                    "SFace",
                    "GhostFaceNet",
                    ]
            recognition = DeepFace.find(img_path=frame_path, db_path="me/",  model_name = models[1],)
            df = pd.DataFrame(recognition[0]) 
            if recognition:
                for person in recognition:
                    take_action(person)
            else:
                logging.info("No match found.")
        except Exception as e:
            logging.error(f"Face recognition failed: {e}")
    else:
        logging.info("No frame downloaded.")

def on_message(client, userdata, message):
    if message.payload.decode() == "start_script":
        main()

client.on_message = on_message
client.connect(mqtt_broker, 1883, 60)
client.subscribe(trigger_topic)
client.loop_start()

# Keep the script running to listen for MQTT messages
try:
    while True:
        pass
except KeyboardInterrupt:
    client.loop_stop()
    client.disconnect()


TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
import requests
import os
import random
import string
import logging
import paho.mqtt.client as mqtt
from deepface import DeepFace
import pandas as pd
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO)

# MQTT Broker details
mqtt_broker = "192.168.1.190"  # Replace with the IP address of your MQTT broker
mqtt_servo_topic = "esp32/servo"
mqtt_trigger_topic = "esp32/trigger"

# Firebase credentials
firebase_admin.initialize_app(cred)
db = firestore.client()

# MQTT Client Setup
client = mqtt.Client()

def download_image(url, save_directory=""):
    try:
        response = requests.get(url)
        response.raise_for_status()

        random_filename = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
        file_extension = "jpg"
        filename = f"{random_filename}.{file_extension}"
        save_path = os.path.join(save_directory, filename)

        with open(save_path, 'wb') as f:
            f.write(response.content)

        logging.info(f"Image downloaded and saved as {save_path}")
        return save_path
    except requests.RequestException as e:
        logging.error(f"Failed to download image: {e}")
        return None

def take_action(found_person):
    logging.info(f"Found person: {found_person['identity']}")
    # Send data to Firebase
    save_to_firebase(found_person)
    # Publish message to move the servo motor
    client.publish(mqtt_servo_topic, "move_servo")

def save_to_firebase(found_person):
    # Add data to Firestore
    data = {
        "name": found_person['identity'],
        "timestamp": datetime.now(),
    }
    db.collection(u'faces').add(data)
    logging.info("Data saved to Firebase")

def main():
    ESP_URL = os.getenv("ESP_URL", "http://192.168.1.236")
    SAVE_DIR = os.getenv("SAVE_DIR", "")

    frame_path = download_image(ESP_URL, SAVE_DIR)
    if frame_path:
        try:
            models = [
                    "VGG-Face", 
                    "Facenet", 
                    "Facenet512", 
                    "OpenFace", 
                    "DeepFace", 
                    "DeepID", 
                    "ArcFace", 
                    "Dlib", 
                    "SFace",
                    "GhostFaceNet",
                    ]
            recognition = DeepFace.find(img_path=frame_path, db_path="me/",  model_name = models[1],)
            df = pd.DataFrame(recognition[0]) 
            if recognition:
                for person in recognition:
                    take_action(person)
            else:
                logging.info("No match found.")
        except Exception as e:
            logging.error(f"Face recognition failed: {e}")
    else:
        logging.info("No frame downloaded.")

def on_message(client, userdata, message):
    if message.payload.decode() == "start_script":
        main()

client.on_message = on_message
client.connect(mqtt_broker, 1883, 60)
client.subscribe(mqtt_trigger_topic)
client.loop_start()

# Publish trigger message to start the script
client.publish(mqtt_trigger_topic, "start_script")

# Keep the script running to listen for MQTT messages
try:
    while True:
        pass
except KeyboardInterrupt:
    client.loop_stop()
    client.disconnect()


TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates